In [11]:
using StatsFuns
gkl_gen(Q, P) = sum(xlogy(p, p/q) for (p, q) in zip(P, Q)) + sum(Q) - sum(P)

gkl_gen (generic function with 1 method)

In [2]:
n = 10^7
P = rand(n)
Q = abs.(randn(n))
gkl_gen(Q, P), gkl_gen(P, Q)

(3.65515495541793e6, 5.469150841913857e6)

In [3]:
gkl_gen(Q, Q)

0.0

In [4]:
gkl_gen(reverse(Q), Q)

5.527102509252604e6

In [12]:
using BenchmarkTools
@benchmark gkl_gen(Q, P)

BechmarkTools.Trial: 38 samples with 1 evaluations.
 Range (min … max):  125.456 ms … 154.701 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     128.830 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   132.459 ms ±   8.811 ms  ┊ GC (mean ± σ):  0.00% ± 0.00%

  █▄ ▄▄ █▁  ▁▄   ▄                                     █         
  ██▆██▆██▁▁██▆▆▁█▁▁▆▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█▆▁▁▁▁▁▆ ▁
  125 ms           Histogram: frequency by time          155 ms <

 Memory estimate: 16 bytes, allocs estimate: 1.

In [6]:
using LoopVectorization

unsafe_xlogy(x, y) = x*log(y)
safe_xlogy(x, y) = iszero(x) ? x : x*log(y)

function gkl_lv(Q, P)
    s = sum(Q) - sum(P)
    @tturbo for i in eachindex(P, Q)
        s += unsafe_xlogy(P[i], P[i]/Q[i])
    end
    s
end

gkl_lv(Q, P)

3.655154955417989e6

In [7]:
@benchmark gkl_lv(Q, P)

BechmarkTools.Trial: 286 samples with 1 evaluations.
 Range (min … max):  15.188 ms …  22.806 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     17.371 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   17.477 ms ± 887.245 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

                  ▂  ▂ ▂▇▅█▅▃▄▁▁                                
  ▃▃▁▃▁▁▁▃▁▁▃▅▆▃▇▆██▅███████████▆▆█▆▆▃▆▃▃▁▃▃▃▁▃▁▃▁▁▁▁▁▃▁▃▃▁▁▃▃ ▃
  15.2 ms         Histogram: frequency by time         20.6 ms <

 Memory estimate: 16 bytes, allocs estimate: 1.

In [8]:
Threads.nthreads()

12